# 14_Celebrities_mobile

## Importing packages

In [88]:
import requests
import zipfile
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

from keras import models, layers, optimizers, callbacks, losses
from keras.preprocessing.image import ImageDataGenerator

In [89]:
base_dir = "../input/14-celebrity-faces-dataset"
celeb14 = os.path.join(base_dir, "14-celebrity-faces-dataset.zip")

with zipfile.ZipFile(celeb14, "r") as z:
    z.extractall('.')

## Model Parameters

In [91]:
img_width, img_height = 224, 224

train_data_dir = os.path.join(data_dir, 'train')
validation_data_dir = os.path.join(data_dir, 'val')
nb_train_samples = 220
nb_validation_samples = 70
epochs = 50
batch_size = 16
numclasses = 14

## Processing images

In [92]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./ 255,
    rotation_range=10,  
    zoom_range=0.1,  
    width_shift_range=0.1,
    height_shift_range=0.1,
    vertical_flip=False,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 220 images belonging to 14 classes.
Found 70 images belonging to 14 classes.


In [93]:
input_shape = (img_width, img_height, 3)

## MobileNet Model

In [94]:
LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

    
def scheduler(epoch):
    if epoch < 4:
        return 0.0005
    elif epoch < 8:
        return 0.0002
    elif epoch < 12:
        return 0.0001
    elif epoch < 16:
        return 0.00005
    elif epoch < 20:
        return 0.00002
    else:
        return 0.00001
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose = True)

In [95]:
def mobile_net():
    model_input = layers.Input(input_shape)
    base_model = keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False,input_shape=input_shape,pooling=False)    
    x = base_model(model_input)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.6)(x)
    model_output = layers.Dense(numclasses, activation='softmax')(x)
    
    model = models.Model(inputs=model_input, outputs=model_output)
    
    return model


model = mobile_net()
model.compile(loss=losses.CategoricalCrossentropy(),
              optimizer=optimizers.rmsprop_v2.RMSprop(learning_rate=0.0005),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[lr_callback])

Epoch 1/50

Epoch 00001: LearningRateScheduler setting learning rate to 1e-05.


In [ ]:
# Get training and test loss histories
val_acc = history.history['val_accuracy']
training_acc = history.history['accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(val_acc) + 1)

# Visualize loss history
plt.figure()
plt.plot(epoch_count, val_acc)
plt.plot(epoch_count, training_acc)
plt.legend(['Validation Accuracy', 'Training Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show();